In [1]:
from transformers import BigBirdTokenizer, BigBirdForSequenceClassification
from transformers import Trainer, TrainingArguments
import torch

print("PyTorch version " + torch.__version__)
print("Num GPUs Available: ", torch.cuda.device_count())
print(torch.cuda.is_available())



c:\Users\ericb\anaconda3\envs\Python10_ML_NLP_Torch\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


PyTorch version 2.0.1
Num GPUs Available:  1
True


In [ ]:

# Assuming you have a DataFrame 'df' with columns 'text' and 'label'

# Initialize the tokenizer
tokenizer = BigBirdTokenizer.from_pretrained('google/bigbird-roberta-base')

# Tokenize the text
encodings = tokenizer(df['text'].tolist(), max_length=4096, truncation=True, padding='max_length', return_tensors='pt')

# Set up the dataset
class EmailDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

dataset = EmailDataset(encodings, df['label'].tolist())

# Initialize the model
model = BigBirdForSequenceClassification.from_pretrained('google/bigbird-roberta-base')

# Define training arguments
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=3,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=10,
    evaluation_strategy="epoch"
)

# Initialize the Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset,
    eval_dataset=dataset
)

# Train the model
trainer.train()
